<a href="https://colab.research.google.com/github/achmfirmansyah/sweet_project/blob/master/Geopython-PythonMachineLearning2020/01_Preparing_data_using_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing the data using Google Earth Engine

Importing the Google Earth Engine Python API and aunthenticate


In [2]:
import ee
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xEtqX5svsQJ3G-3ZZ5-i0l72lZAr8pruAK5xRKGwzr0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4QGvMuSB-2nE7X_ADfKG4Ed66SxpdtfH3kuwb-n-NpCBrgAE_1tm7AU

Successfully saved authorization token.


In [3]:
ee.Initialize()

## Mosaic the reference imagery
<p>For built up references, i use the Global Human Settlement Layer (you can download the data from this link https://ghsl.jrc.ec.europa.eu/datasets.php). After you download the Tiff data, upload the tiff to your asset so you can combine the reference dataset with other public imagery provided by Google Earth Engine. You can follow step in this link for uploading the imagery: https://developers.google.com/earth-engine/guides/image_upload  <p>

<p>Unfortunately the data is clipped by several region, so first i do the mosaic of the imagery into one Tiff.</p>


In [4]:
# Call all the image dataset you download
image1=ee.Image('users/achmfirmansyah/GHSL_30/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_20_11')
image2=ee.Image('users/achmfirmansyah/GHSL_30/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_21_11')
image3=ee.Image('users/achmfirmansyah/GHSL_30/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_21_12')
image4=ee.Image('users/achmfirmansyah/GHSL_30/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_22_11')
image5=ee.Image('users/achmfirmansyah/GHSL_30/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_22_11')

# Here i mosaic the data into one and rename the band names into BU_class
GHSL_mosaic=ee.ImageCollection([image1,image2,image3,image4,image5]).mosaic().select('b1').rename('BU_class')

## Define the geometry boundary

<p>The geometry boundary is used here for clipping the imagery into one specific boundary for the analysis. This step is used for focusing the analysis and reduce the size of imagery.</p>


In [5]:
# Here i define the geometry boundary of my study area.
jabodetabek=[[106.347,-5.906],[107.300,-6.823]]

## Process the public imagery used as classification feature
This section is about how i download the public imagery for my research, get the median composite value, and calculate the indices.
The public imagery i used is Landsat 8 and VIIRS. Landsat 8 is used for reflect the surface characteristics and VIIRS is used for reflect human behavior. From Landsat 8, i calculate the NDVI (Normalized Difference of Vegetation Index), MNDWI (Modified Normalized Differnces of  Watery Index), NDBI (Normalized Difference of Built Index), SAVI (Soil Adjusted Vegetation Index). And i used the Avg-Radiance value from VIIRS DNB imagery.


In [9]:
# Landsat 8
landsat_8_2014=ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2014-01-01', '2014-12-31').select(['B4','B5','B6','B3']).reduce(ee.Reducer.median())
# Calculate in indices
ndvi_landsat_8=landsat_8_2014.normalizedDifference(['B5_median','B4_median']).rename('NDVI_landsat_8')
ndbi_landsat_8=landsat_8_2014.normalizedDifference(['B6_median','B5_median']).rename('NDBI_landsat_8')
mndwi_landsat_8=landsat_8_2014.normalizedDifference(['B3_median','B6_median']).rename('MNDWI_landsat_8')
savi_landsat_8=landsat_8_2014.select('B5_median').subtract(landsat_8_2014.select('B4_median')).divide(landsat_8_2014.select('B5_median').add(landsat_8_2014.select('B4_median')).add(0.5)).multiply(1.5).rename('SAVI_landsat_8')
landsat_8_oke=ee.Image.cat(ndvi_landsat_8,ndbi_landsat_8,mndwi_landsat_8,savi_landsat_8).toDouble()
# VIIRS
viirs_2014=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2014-01-01','2014-12-31')
viirs_2014=viirs_2014.select('avg_rad')
median_viirs_2014=viirs_2014.reduce(ee.Reducer.median())

### Combine the imagery from references class and the classification feature
For combine the data, both of imagery must be in same projection. After all of the projection of the imagery is same, we can combine the data 

In [10]:
# get the projection
project1=median_viirs_2014.projection();
# project
GHSL_mosaic=GHSL_mosaic.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
landsat_oke=landsat_8_oke.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
image_compiled=ee.Image.cat(GHSL_mosaic,landsat_oke,median_viirs_2014).toDouble()

### Export the imagery to the drive for next Classification

In [11]:
ee.batch.Export.image.toDrive(
    image=image_compiled,
    description='compiled_GHSL_30_2014_',
    folder= 'GeoPythonML_',
    scale=30,
    crs='EPSG:4326',
    region=jabodetabek,
    fileFormat='GEO_TIFF').start()